<a href="https://colab.research.google.com/github/kjspring/stress-detection-wearable-devices/blob/main/WESAD_data_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kjspring/stress-detection-wearable-devices.git

Cloning into 'stress-detection-wearable-devices'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 47 (delta 19), reused 36 (delta 11), pack-reused 0
Unpacking objects: 100% (47/47), 261.87 KiB | 1.65 MiB/s, done.


In [2]:
!git clone https://github.com/kjspring/pyEDA.git

Cloning into 'pyEDA'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 290 (delta 151), reused 203 (delta 87), pack-reused 0
Receiving objects: 100% (290/290), 11.30 MiB | 9.45 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [3]:
! git clone https://github.com/neuropsychology/NeuroKit

Cloning into 'NeuroKit'...
remote: Enumerating objects: 99906, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 99906 (delta 43), reused 287 (delta 37), pack-reused 99605
Receiving objects: 100% (99906/99906), 3.10 GiB | 26.75 MiB/s, done.
Resolving deltas: 100% (49281/49281), done.
Updating files: 100% (613/613), done.


In [4]:
! pip install neurokit2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00


In [3]:
# Exploratory data analysis of dataset

import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import pickle
from pyEDA.main import *
import neurokit2 as nk

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
! ls /content/drive/MyDrive/stress-prediction

cvxEDA.py  data  data_wrangling.py  files  __pycache__	WESAD


In [6]:
#! cd /content/drive/MyDrive/stress-prediction/ # Main file directory for this notebook on Google Drive
os.chdir('/content/drive/MyDrive/stress-prediction')
# absolute path of data directory
PATH = os.path.join(os.path.abspath(os.getcwd()), 'data') # Path of data folder on Google Drive

'/content/drive/MyDrive/stress-prediction/data/1. processed'

## Open and clean up raw data
The data for swell and wsead is stored in seperate folders. Within each of these is a subfolder holding the eda, rri, and Labels folders

In [ ]:
original_dir = os.path.join(PATH, 'original')
org_WESAD_dir = os.path.join(original_dir, 'WESAD')
#process_eda_swell_files = glob.glob(os.path.join(org_WESAD_dir, "*.pkl"))
print(org_WESAD_dir)

In [ ]:
class SubjectData:
# https://github.com/WJMatthew/WESAD/blob/master/data_wrangling.py
    def __init__(self, main_path, subject_number):
        #print(main_path)
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        #self.wrist_keys = ['EDA']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'RSP': self.data['signal']['chest']['Resp']})
        #data.update({'ECG' : self.data['signal']['chest']['ECG']})
        #data.update({'labels': self.labels})
        return data
    
    def get_wrist_eda_data(self):
        data = self.data['signal']['wrist']['EDA']
        return data

    def get_chest_data(self):
        return self.data['signal']['chest']

    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.get_wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results


In [ ]:
def subject_data_import(directory):
  import regex as re

  regex = re.compile(r'\d+')
  subject_li = []
  ext = '.pkl' # pickle extension
  # Walk through data directory and only return with specific extension
  for path, dirc, files in os.walk(directory):
    for name in files:
      if name.endswith(ext):
        #print(name) # print file name
        #print(path)
        subject_number = regex.findall(name) # Extract the subject number
        #print(subject_number[0])
        #print(type(int(subject_number[0])))
        subject_li.append(SubjectData(directory, int(subject_number[0]))) # Create SubjectData object

  return subject_li

In [ ]:
wesad_sd_li = subject_data_import(org_WESAD_dir)

#wesad_raw = pd.read_pickle(org_WESAD_dir+"/S3/S3.pkl")

## Dictionary analysis


In [ ]:
# Extract wrist data
wesad_data_li = [ f.get_wrist_data() for f in wesad_sd_li ]

In [ ]:
# Grab only subject 4
S4 = SubjectData(org_WESAD_dir, 4)
S4_wrist = S4.get_wrist_data() # get wrist data

In [ ]:
wesad_data_li[0]['EDA']

In [ ]:
# pyEDA
m, wd, eda_clean = process_statistical(wesad_data_li[0], 
                                       use_scipy=True, 
                                       sample_rate=128, 
                                       new_sample_rate=40, 
                                       segment_width=600, 
                                       segment_overlap=0)

In [ ]:
# pyEDA autoencoder
prepare_automatic(wesad_data_li[0]/1, sample_rate=128, new_sample_rate=40, k=32, epochs=100, batch_size=10)

In [ ]:
#print(f'Lenght of ACC: {len(S3_wrist["ACC"])}')
#print(f'Lenght of BVP: {len(S3_wrist["BVP"])}')
#print(f'Lenght of EDA: {len(S3_wrist["EDA"])}')
#print(f'Lenght of TEMP: {len(S3_wrist["TEMP"])}')
#print(f'Lenght of Resp: {len(S3_wrist["Resp"])}')
#print(f'Lenght of labels: {len(S3_wrist["labels"])}')

In [ ]:
# Neurokit2

# process the data (filter, find peaks, ect.)
processed_data, info = nk.bio_process(rsp=wesad_data_li[0]['RSP'],
                                      eda=wesad_data_li[0]['EDA'],
                                      sampling_rate=100)

In [ ]:
results = nk.bio_analyze(processed_data, sampling_rate=100)

In [ ]:
results.info()

In [ ]:
# Visualize
# Simulate
eda1 = nk.eda_simulate(duration=10, scr_number=1, drift=-0.01, noise=0.05)
eda3 = nk.eda_simulate(duration=10, scr_number=3, drift=-0.01, noise=0.01)
eda3_long = nk.eda_simulate(duration=10, scr_number=3, drift=-0.1, noise=0.01)

# Visualize
eda_df = pd.DataFrame({"EDA1": eda1,
                       "EDA3": eda3,
                       "EDA3_Longer": eda3_long})
nk.signal_plot(eda_df, subplots=True)

In [ ]:
processed_data['EDA_Clean'].describe()

In [ ]:
S4_wrist['EDA']

In [ ]:
processed_data_S4_eda, info = nk.bio_process(eda=S4_wrist['EDA'],
                                      sampling_rate=100)
processed_data_S4_eda['EDA_Clean']

In [ ]:
nk.signal_plot(processed_data_S4_eda['EDA_Clean'])

In [ ]:
len(S4_wrist['EDA'])